In [1]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import eigsh
from scipy.spatial import distance
from math import dist
from math import floor

from numpy import array, array_equal, allclose

import pandas as pd

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


from weights import KNN
from weights import proximity

from accuracy import KNN_acc

In [225]:
def train_test(train: pd.DataFrame, test: pd.DataFrame, num1: int, num2: int, ker="rbf"):
    y_train = train["label"]
    x_train = train.loc[:, [x for x in train.columns if x != "label"]]

    y_test = test["label"]
    x_test = test.loc[:, [x for x in train.columns if x != "label"]]

    pca = PCA()
    pca.fit(x_train)
    x_pca = pca.transform(x_train)

    var = pca.explained_variance_ratio_
    sum_var = 0
    i_count = 0
    for i in range(len(var)):
        sum_var += var[i]
        if sum_var >= 0.8:
            i_count = i
            break

    np.sum(var[:i_count+1])

    x_pca = x_pca[:,:(i_count+1)]

    def collect_nums(x: pd.DataFrame, num1: int, num2: int):
        i_list = x.index[(x['label']==num1)|(x['label']==num2)].tolist()
        return i_list

    num_indices_0_train = collect_nums(train,num1,num2)
    num_indices_0_test = collect_nums(test,num1,num2)
    
    x_train = x_train.to_numpy()
    x_train = x_train[num_indices_0_train,:]
    x_pca = x_pca[num_indices_0_train,:]
    y_train = y_train[num_indices_0_train].to_numpy()
    y_train[y_train==num1] = -1
    y_train[y_train==num2] = 1

    # scale for the rbf
    if ker == "rbf":
        scaler = StandardScaler().fit(x_train)
        x_train_scaled = scaler.transform(x_train)
        scaler = StandardScaler().fit(x_pca)
        x_pca_scaled = scaler.transform(x_pca)

        return x_train_scaled, y_train, x_pca_scaled

    else:
        return x_train, y_train, x_pca

In [221]:
train = pd.read_csv("../Data/mnist_train.csv")
test = pd.read_csv("../Data/mnist_test.csv")

In [226]:
X, y, X_pca = train_test(train, test, 0, 1,"rbf")
X = X[:500,:]
y = y[:500]
X_pca = X_pca[:500,:]

In [239]:
#Global variables
size = 30
k = 6 #was 3
tau = 1  #was 1
alpha = 2 #was 2
lossf = "probit"

dists = []
for x1 in X:
	for x2 in X:
		dists += [dist(x1,x2)]
dists.sort()

mult = 1 #brought this down order of 1000
# mult = 1
# gamma = dists[floor(len(dists)/4)]*mult
# gamma = 0.385 #gamma=0.4 kinda works
# gamma = 0.07
gamma = 0.04
# gamma=0.01
# print(gamma)

rbf = lambda x1, x2: np.exp(gamma**2*-0.5*dist(x1,x2)**2)

lamb = (tau**(2*alpha))/2

In [240]:
kvals_keep = np.random.randint(0,y.shape[0],size=size)
kvals = y[kvals_keep]

In [241]:
L,W = KNN(X,len(y),k,rbf)

Computing distance matrix
Finished computing graph laplacian


In [242]:
np.linalg.eigh(L)[0]

array([-3.31808911e-15,  1.71311673e-16,  8.42625508e-09,  8.70259055e-09,
        2.27330159e-04,  1.45922591e-03,  1.02694810e-02,  2.78753629e-02,
        1.22554870e-01,  2.79525182e-01,  3.68731655e-01,  4.48544691e-01,
        5.26223713e-01,  5.61065789e-01,  6.28882578e-01,  6.75601770e-01,
        7.04988379e-01,  7.49244193e-01,  7.95526606e-01,  8.11070142e-01,
        8.20274026e-01,  8.97946601e-01,  9.53058619e-01,  1.02619215e+00,
        1.05831101e+00,  1.13683437e+00,  1.17165044e+00,  1.19339506e+00,
        1.32648497e+00,  1.36191928e+00,  1.37510481e+00,  1.55230821e+00,
        1.58659722e+00,  1.60167616e+00,  1.64422089e+00,  1.71959971e+00,
        1.73712176e+00,  1.77101942e+00,  1.81653708e+00,  1.87055155e+00,
        1.88239094e+00,  1.92025672e+00,  1.95058521e+00,  1.98259651e+00,
        2.01523371e+00,  2.05310573e+00,  2.10331292e+00,  2.16750942e+00,
        2.17431441e+00,  2.24135416e+00,  2.34257044e+00,  2.34848823e+00,
        2.39319675e+00,  

In [243]:
acc = KNN_acc(X, y, kvals, k, tau, alpha, lossf, rbf)

entering in KNN function
Computing distance matrix
Finished computing graph laplacian
Starting the minimization
After minimizing


In [244]:
acc

0.896

In [245]:
lossr = "regression"
acc_2 = KNN_acc(X, y, kvals, k, tau, alpha, lossr, rbf)
acc_2

entering in KNN function
Computing distance matrix
Finished computing graph laplacian
Starting the minimization
After minimizing


0.908

In [246]:
unif = lambda x1, x2 : 1
acc_3 = KNN_acc(X, y, kvals, k, tau, alpha, lossf, unif)
acc_3

entering in KNN function
Computing distance matrix
Finished computing graph laplacian
Starting the minimization
After minimizing


0.85

In [247]:
acc_4 = KNN_acc(X, y, kvals, k, tau, alpha, lossf, unif)
acc_4

entering in KNN function
Computing distance matrix
Finished computing graph laplacian
Starting the minimization
After minimizing


0.85